In [42]:
target_couplings = ["cw", "cwtil", "chw", "chwtil", "chwb", "chwbtil"]
target_couplings = [s.lower() for s in target_couplings]

In [107]:
def modify_file (fname, outname) :
    f = open(fname, "r")
    f_out = open(outname, "w")
    coupling_found, value_found = False, False
    coupling_name, is_target = "", False
    coupling_names = []
    for line in f :
        if "Coupling(name = " in line : 
            coupling_found = True
            f_out.write(line)
            coupling_name = line[line.find("name = ")+7:line.find(",")]
            continue
        if coupling_found == False and value_found == False :
            f_out.write(line)
            continue
        coupling_found = False
        if "value = " in line : 
            value_found = True
            value = line[line.find("value = ")+9:-1]
            value = value[:value.find("'")]
            value = value.lower()
            for s in target_couplings :
                if value.find(s) < 0 : continue
                is_target = True
            if is_target :
                coupling_names.append(coupling_name.replace("'","").replace("GC_",""))
                print(coupling_name)
                print(value)
                print()
            f_out.write(line)
            continue
        if value_found == False :
            f_out.write(line)
            continue
        value_found = False
        if is_target is False and "NP" in line :
            pos = line.find("{'NP':")
            line = line[:pos+6] + "100" + line[pos+7:]
        is_target = False
        f_out.write(line)
    return coupling_names

In [108]:
coupling_names = modify_file("couplings_original.py", "couplings.py")

'GC_33'
(4*chw*complex(0,1))/lambdasmeft**2

'GC_34'
(4*chwtil*complex(0,1))/lambdasmeft**2

'GC_52'
(6*cth*cw*complex(0,1))/lambdasmeft**2

'GC_53'
(2*cth*cwtil*complex(0,1))/lambdasmeft**2

'GC_54'
(6*cth*cw*ee*complex(0,1))/lambdasmeft**2

'GC_55'
(-2*cth*cwtil*ee*complex(0,1))/lambdasmeft**2

'GC_244'
(6*cw*ee*complex(0,1))/(lambdasmeft**2*sth)

'GC_245'
(-6*cth**2*cw*ee*complex(0,1))/(lambdasmeft**2*sth)

'GC_246'
(-2*cwtil*ee*complex(0,1))/(lambdasmeft**2*sth)

'GC_247'
(2*cth**2*cwtil*ee*complex(0,1))/(lambdasmeft**2*sth)

'GC_250'
(6*cw*complex(0,1)*sth)/lambdasmeft**2

'GC_251'
(2*cwtil*complex(0,1)*sth)/lambdasmeft**2

'GC_252'
(-6*cw*ee*complex(0,1)*sth)/lambdasmeft**2

'GC_253'
(2*cwtil*ee*complex(0,1)*sth)/lambdasmeft**2

'GC_256'
(4*chw*cth**2*complex(0,1))/lambdasmeft**2 + (4*chwb*cth*complex(0,1)*sth)/lambdasmeft**2 + (4*chb*complex(0,1)*sth**2)/lambdasmeft**2

'GC_257'
(-2*chwtil*cth**2*complex(0,1))/lambdasmeft**2 - (2*chwbtil*cth*complex(0,1)*sth)/lambdasmeft**2 - (2

In [109]:
for n in coupling_names :
    print(n)

33
34
52
53
54
55
244
245
246
247
250
251
252
253
256
257
258
259
260
261
270
271
272
273
274
275
281
282
283
284
295
296
297
299
300
301
320
321
322
323
324
325
326
327
328
338
339
340
341
342
343
344
345
346
347
350
351
352
353
354
355
358
359


In [110]:
class Vertex :
    def __init__ (self, name="", particles="", couplings="") :
        self.name      = name
        self.particles = particles
        self.couplings = couplings

In [133]:
def get_couplings (line) :
    ret = []
    idx = 0
    coupling = ""
    record_coupling = False
    while idx < len(line) :
        if idx < len(line)-3 and line[idx:idx+3] == "GC_" :
            coupling = ""
            record_coupling = True
            idx = idx + 3
            continue
        if record_coupling is True :
            if line[idx] == "," :
                record_coupling = False
                if coupling in coupling_names : 
                    ret.append(coupling)
                coupling = ""
            else :
                coupling = coupling + line[idx]
        idx = idx + 1
    return ret
    

In [137]:
def get_vertices (fname) :
    vertices = []
    name, particles = "", ""
    for line in open(fname, "r") :
        if "Vertex(" in line :
            name = line[line.find("name = '")+8:line.find("',")]
            continue
        if "particles = " in line :
            particles = line[line.find("particles = [ ")+14:line.find("],")].replace("P.","").replace("__minus__","").replace("__plus__","").replace("__tilde__","")
            continue
        if "couplings" not in line : continue
        couplings = get_couplings(line)
        vertices.append(Vertex(name, particles, couplings))
        name      = ""
        particles = ""
    return vertices   

In [138]:
vertices = get_vertices("vertices.py")

In [144]:
for v in vertices :
    if len(v.couplings) == 0 : continue
    print(f"{v.name.ljust(6)} with particles  "+f"[[{v.particles}]]".ljust(20)+"  depends on couplings  " + "  ".join(v.couplings))

V_7    with particles  [[a, a, H ]]          depends on couplings  325
V_9    with particles  [[a, a, H, H ]]       depends on couplings  261
V_13   with particles  [[W, W, H, H ]]       depends on couplings  34  33
V_15   with particles  [[W, W, H ]]          depends on couplings  271  270
V_17   with particles  [[a, W, W ]]          depends on couplings  297  300  251  250
V_19   with particles  [[a, Z, H ]]          depends on couplings  324
V_21   with particles  [[a, Z, H, H ]]       depends on couplings  260  259
V_22   with particles  [[Z, Z, H, H ]]       depends on couplings  257  256
V_24   with particles  [[Z, Z, H ]]          depends on couplings  321  320
V_26   with particles  [[W, W, Z ]]          depends on couplings  301  296  53  52
V_59   with particles  [[a, W, W, H ]]       depends on couplings  275  283  272
V_60   with particles  [[a, a, W, W ]]       depends on couplings  253  252
V_62   with particles  [[a, W, W, Z ]]       depends on couplings  55  54
V_64   w